In [2]:
# #importing libraries
# import warnings
# warnings.filterwarnings("ignore")
from tensorflow.keras.preprocessing import text
from tensorflow.keras.utils import to_categorical as np_utils
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.utils import pad_sequences
import numpy as np
import pandas as pd

In [2]:
#taking random sentences as data
data = """Deep learning (also known as deep structured learning) is part of a broader family of machine learning methods based on artificial neural networks with representation learning. Learning can be supervised, semi-supervised or unsupervised. 
Deep-learning architectures such as deep neural networks, deep belief networks, deep reinforcement learning, recurrent neural networks, convolutional neural networks and Transformers have been applied to fields including computer vision, speech recognition, natural language processing, machine translation, bioinformatics, drug design, medical image analysis, climate science, material inspection and board game programs, where they have produced results comparable to and in some cases surpassing human expert performance.
"""
dl_data = data.split()

In [3]:
#tokenization
tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(dl_data)
word2id = tokenizer.word_index

word2id['PAD'] = 0
id2word = {v:k for k, v in word2id.items()}
wids = [[word2id[w] for w in text.text_to_word_sequence(doc)] for doc in dl_data]

vocab_size = len(word2id)
embed_size = 100
window_size = 2 

print('Vocabulary Size:', vocab_size)
print('Vocabulary Sample:', list(word2id.items())[:10])

Vocabulary Size: 75
Vocabulary Sample: [('learning', 1), ('deep', 2), ('networks', 3), ('neural', 4), ('and', 5), ('as', 6), ('of', 7), ('machine', 8), ('supervised', 9), ('have', 10)]


In [4]:
#generating (context word, target/label word) pairs
def generate_context_word_pairs(corpus, window_size, vocab_size):
    context_length = window_size*2
    for words in corpus:
        sentence_length = len(words)
        for index, word in enumerate(words):
            context_words = []
            label_word   = []            
            start = index - window_size
            end = index + window_size + 1
            
            context_words.append([words[i] 
                                 for i in range(start, end) 
                                 if 0 <= i < sentence_length 
                                 and i != index])
            label_word.append(word)

            x = pad_sequences(context_words, maxlen=context_length)
            y = np_utils(label_word, vocab_size)
            yield (x, y)
            
i = 0
for x, y in generate_context_word_pairs(corpus=wids, window_size=window_size, vocab_size=vocab_size):
    if 0 not in x[0]:
        # print('Context (X):', [id2word[w] for w in x[0]], '-> Target (Y):', id2word[np.argwhere(y[0])[0][0]])
    
        if i == 10:
            break
        i += 1

In [5]:
#model building
import tensorflow.keras.backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Lambda

cbow = Sequential()
cbow.add(Embedding(input_dim=vocab_size, output_dim=embed_size, input_length=window_size*2))
cbow.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(embed_size,)))
cbow.add(Dense(vocab_size, activation='softmax'))
cbow.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])

print(cbow.summary())

# from IPython.display import SVG
# from keras.utils.vis_utils import model_to_dot

# SVG(model_to_dot(cbow, show_shapes=True, show_layer_names=False, rankdir='TB').create(prog='dot', format='svg'))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 100)            7500      
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 75)                7575      
                                                                 
Total params: 15,075
Trainable params: 15,075
Non-trainable params: 0
_________________________________________________________________
None


In [6]:
cbow.fit(x, y, epochs=10, verbose=1)

Epoch 1/10
1/1 [==============================] - 1s 1s/step - loss: 4.3501 - accuracy: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 0s 6ms/step - loss: 4.3339 - accuracy: 0.0000e+00
Epoch 3/10
1/1 [==============================] - 0s 7ms/step - loss: 4.3176 - accuracy: 0.0000e+00
Epoch 4/10
1/1 [==============================] - 0s 7ms/step - loss: 4.3013 - accuracy: 0.0000e+00
Epoch 5/10
1/1 [==============================] - 0s 6ms/step - loss: 4.2851 - accuracy: 0.0000e+00
Epoch 6/10
1/1 [==============================] - 0s 8ms/step - loss: 4.2688 - accuracy: 0.0000e+00
Epoch 7/10
1/1 [==============================] - 0s 6ms/step - loss: 4.2525 - accuracy: 1.0000
Epoch 8/10
1/1 [==============================] - 0s 7ms/step - loss: 4.2361 - accuracy: 1.0000
Epoch 9/10
1/1 [==============================] - 0s 5ms/step - loss: 4.2198 - accuracy: 1.0000
Epoch 10/10
1/1 [==============================] - 0s 6ms/step - loss: 4.2034 - accuracy: 1.0000


In [7]:
weights = cbow.get_weights()[0]
weights = weights[1:]
print(weights.shape)

pd.DataFrame(weights, index=list(id2word.values())[1:]).head()

(74, 100)


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
deep,0.049555,0.013263,-0.037127,0.015522,0.024142,-0.020744,-0.018974,0.014090,-0.027763,0.029944,...,-0.022640,0.049408,-0.047843,-0.021477,-0.041879,0.002027,0.031538,-0.026211,0.028766,0.011593
networks,-0.043153,-0.015343,-0.044785,-0.047414,-0.041467,-0.013872,0.047790,-0.015439,-0.006875,0.036161,...,-0.033633,0.036782,-0.045058,-0.049737,0.003334,-0.041389,-0.049591,0.034766,-0.039074,-0.013693
neural,0.011656,0.008567,-0.001324,-0.040477,-0.009425,-0.017866,-0.000441,-0.010358,-0.042919,0.020014,...,-0.033135,-0.011705,0.009918,0.047956,0.047155,0.018235,0.017842,-0.014145,0.003705,0.013786
and,-0.003117,0.030722,0.041774,-0.015686,0.020573,0.041941,0.003022,-0.011052,-0.010028,-0.044554,...,0.015895,0.048282,0.021178,0.036694,-0.002875,0.045490,-0.008114,-0.013406,0.049373,-0.033712
as,0.005712,0.002244,0.026866,-0.040194,-0.046970,-0.000044,0.007143,0.046381,0.018130,-0.027704,...,0.042003,-0.029192,0.020552,0.012791,0.014633,0.008141,0.001320,0.034206,0.030691,0.037772


In [8]:
from sklearn.metrics.pairwise import euclidean_distances

distance_matrix = euclidean_distances(weights)
print(distance_matrix.shape)

similar_words = {search_term: [id2word[idx] for idx in distance_matrix[word2id[search_term]-1].argsort()[1:6]+1] 
                   for search_term in ['networks']}

similar_words

(74, 74)


{'networks': ['recurrent', 'image', 'in', 'methods', 'speech']}